In [1]:
a=[1,2,3,4,5,6]
for i in range(0,len(a),2):
    print(i)

0
2
4


In [ ]:
# long crosshatch plots SUBPLOTS FOR FINDING THE THRESHOLD


path= '/Users/estepark/Documents/week6_fri/Data/'
# row 500, 1275, 1350
def subPlotPlotLong(path, plotRows=[]):
    pixelIntensities = []
    fftAmps = []
    fnames = []
    count = []
    for fname in (glob.glob(path+'/*.csv')) :
        Lv_csv = pd.read_csv(fname, skiprows=[0], header=None)
        for i in range(0,1500):
            # find element's index corresponding to u'
            if ( Lv_csv[0][i] =="u'" ): 
                # save all Lv matrix values until u' index
                Lvindex = int(np.where(Lv_csv == Lv_csv[0][i])[0] ) 
                break     # as soon as matched index valid, exit & search no further
        # concatenated Lv matrix that excludes u' and v' matrix in csv file
        Lv = np.array(Lv_csv[0: Lvindex], dtype="float32" ) 
        """
        INPUT == Short cross hatches
        FN == Short cross hatch search
        OUTPUT == ALL THE SUBPLOTS AND SHIET
        """
        # split the name of the csv as plt title
        spltName = fname.split('/')[-1][:-4] 
        fnames.append(spltName)
        plt.figure()
        imgLv = np.array(Lv, dtype = "uint8") #cv2 for 8u and 32f images in function 'bilateralFilter'
        imgLvcopy = imgLv.copy()
        #cv.imread(Lv.x1)
        plt.title("Lv from csv file converted to image \n {}".format(spltName))
        plt.imshow(np.array(Lv), cmap="gray")
        plt.show()

        # provide Scharr kernel
        kern = [[0, 1,  0], [1, 0, -1],[ 0, -1 ,0]]
        imgrayLv = cv.bilateralFilter(imgLv, 20, 75, 75)
        diag_imgLv= signal.convolve2d(imgrayLv, np.array(kern) , boundary ='symm', mode='same')
        plt.figure()
        plt.imshow(diag_imgLv, cmap = "gray")
        plt.title("Scharr + LPF \n {}".format(spltName))
        plt.show()

        # scale back to [0,255], becuase convolve2d changes ranges [-2,+2]
        diag_cpy = (((diag_imgLv.copy()+2))/4*255).astype(np.uint8) #bc min=-2 and max=+2
        #print('max = {}, min = {}'.format(np.max(diag_cpy), np.min(diag_cpy)))  #confirm min/max
        #print(' diag shape = ',diag_cpy.shape) 
        edged = cv.Canny(diag_cpy, 160, 230)

        #sobelx = cv.Sobel(diag_cpy,cv.CV_64F,0,1,ksize=5)
        #plt.figure()
        #plt.imshow(diag_cpy)
        #plt.title('diag_cpy scaled to 0,255')
        #plt.show()
        #==================================================
        # perform short crosshatch # FFT scores vs. # row slice, around row #1200 should see period 
        #==================================================
        samplingFreq = 100
        samplingInterval = 1 / samplingFreq
        beginTime = 0
        print(imgLv.shape)
        """
        numRows == len of col is a 1360(height)x1020(width), then 1360==len of col == #of rows
        """
        numRows = imgLv.shape[0] 
        numCols = imgLv.shape[1]
        # for loop for short crosshatch is endTime = numCol
        #freqRowSamps = np.linspace(0, numCols, numCols) # for short crosshatch
        freqRowSamps = np.linspace(0, numRows, numRows) # for short crosshatch
        endTime = numRows
        teim = np.arange(beginTime,endTime, 1)
        countPeaks = [] 
        score = []
        print("numrows: {}" .format(imgLv.shape[0]))
        print("numCols: {}" .format(imgLv.shape[1]))

        filePixelIntensities = []
        fileFftAmps = []
        for i in range(numCols):
        #for i in plotRows: 
            # for every i'th row, slice all columns [i,:]: => and append into amplitude 
            #amplitude = imgrayLv[i,:] # ampltiude == # of cols, for short crosshatch
            amplitude = imgrayLv[:,i] # ampltiude == # of cols, for short crosshatch
            filePixelIntensities.append(amplitude)
            #================================================================================
            #time domain representation for CrossHatch periodic signal
            #================================================================================
            #plt.figure()
            #plt.title('Row #%s' % (i) + " Spatial Intensity vs. Pixel distance \n{}".format(spltName) )
            #plt.xlabel('Pixel Distance along x-axis ')
            #plt.ylabel('Pixel intensity of Lv mat [Nits]')
            #plt.plot(amplitude)

            #=================================================================================
            #frequency domain of short CrossHatch
            # ---->https://www.oreilly.com/library/view/elegant-scipy/9781491922927/ch04.html
            #=================================================================================
            #plt.figure()
            X = fftpack.fft(amplitude)
            fileFftAmps.append(np.abs(X))
            amplitude1 = fftpack.fftfreq(len(amplitude))*samplingFreq
        
            #plt.title('Row #%s' % (i) + " Abs FFT Magnitude vs. Frequency \n{}".format(spltName) )
            #plt.xlabel('Frequency along x-axis [cycles/image]')
            #plt.ylabel('Abs amplitude FFT')
            #plt.xlim(-1, samplingFreq / 2)
            #plt.ylim(-1, 4000)
            #plt.stem(freqRowSamps, np.abs(X), use_line_collection = True) 

            #=================================================================================
            #count number of peaks that surpass that threshold, dont xlim ylim
            #plt.subplots_adjust ( hspace = 3)     
            # https://pythonawesome.com/overview-of-the-peaks-dectection-algorithms-available-in-python/
            #=================================================================================
            freqPeaks = scipy.signal.find_peaks( np.abs(X) , distance=1, width=0, height=[250, 3000] )
            
            indexPeaks = freqPeaks[0]
            heightPeaks = freqPeaks[1]['peak_heights']
           

            if heightPeaks.size > 0:
                #plt.stem(indexPeaks,heightPeaks, use_line_collection=True)
                countPeaks.append(heightPeaks)
                #plt.show()
            else:
                countPeaks.append([0])

        countPeaks = np.array(countPeaks) 
        isCrossHatch = np.hstack(countPeaks) #flattend last of countPeaks
        
        score = sum(isCrossHatch)
        print("score is {}".format(score))
        sumCols = [np.sum(cp) for cp in countPeaks]
        count.append(sumCols)


    plt.plot(count[0],'b-')
    plt.plot(count[1],'r-')
    plt.plot(count[2],'g-')
    plt.xlabel('pixel location along short x-axis')
    plt.ylabel('Sum of FFT magnitude')
    plt.title('sum of FFT mag vs. pixel location x-axis \n{}'.format(spltName))
    plt.legend((line1, line2, line3), 'a','b','c',loc="upper right")
    plt.show()
"""
        # perform long crosshatch
        pixelIntensities.append(np.array(filePixelIntensities))
        fftAmps.append(np.array(fileFftAmps))
    
    #pixelIntensities = np.array(pixelIntensities)
    #fftAmps = np.array(fftAmps)
    
    plt.figure()
    fig, ax = plt.subplots(len(plotRows), 2,figsize=(20,25))
    for j, row in enumerate(plotRows): 
        for k in range(len(fnames)): 
            ax[j, 0].plot(pixelIntensities[k][j], label='{}'.format(fnames[k])) 
            ax[j, 1].stem(fftAmps[k][j], label='{}'.format(fnames[k]), use_line_collection=True, markerfmt='C{}o'.format(k),linefmt='C{}-'.format(k))
        ax[j,0].set_title('Row #{}'.format(row) + " Spatial Intensity vs. Pixel distance ", fontsize=20 )
        ax[j,0].set_xlabel('Pixel Distance along x-axis ' , fontsize=18)
        ax[j,0].set_ylabel('Pixel intensity of Lv mat [Nits]',fontsize=18)

        ax[j,0].legend()
        
        ax[j,1].set_title('Row #{}'.format(row) + " Abs FFT Magnitude vs. Frequency ", fontsize=20)
        ax[j,1].set_xlabel('Frequency along x-axis [cycles/image]',fontsize=18)
        ax[j,1].set_ylabel('Abs amplitude FFT',fontsize=18)
        ax[j,1].set_xlim(-1, samplingFreq / 2)
        ax[j,1].set_ylim(-1, 2000)

        ax[j,1].legend()
    fig.tight_layout()
    plt.show()


    

"""
path= '/Users/estepark/Documents/week6_fri/Data/'
# row 500, 1275, 1350
subPlotPlotLong(path, plotRows=[0, 100, 700])




In [13]:
import numpy as np
import math
a= [1,10,3,100]
a=np.array(a)
c =map(lambda x: 10*math.log10(x) , a)

In [8]:
print("{}".format(c))

In [10]:
10*np.log10(a)

array([ 0.        , 10.        ,  4.77121255, 20.        ])

In [14]:
c

In [15]:
list(c)

[0.0, 10.0, 4.771212547196624, 20.0]